In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
train_data_path = "dataset/train.csv"
test_data_path = "dataset/test.csv"

train_dataframe = pd.read_csv(train_data_path)
test_dataframe = pd.read_csv(test_data_path)

#train_dataframe.head(5)

In [ ]:
label_encoder = LabelEncoder()

train_dataframe['Gender'] = label_encoder.fit_transform(train_dataframe['Gender'])
train_dataframe = pd.get_dummies(train_dataframe, columns=['Geography'])
x = train_dataframe.drop(['Exited','Surname','CustomerId'], axis=1)
y = train_dataframe['Exited']

#x_train.head(10)
#y_train.head(10)

test_dataframe['Gender'] = label_encoder.fit_transform(test_dataframe['Gender'])
test_dataframe = pd.get_dummies(test_dataframe, columns=['Geography'])
x_test = test_dataframe.drop(['Surname','CustomerId'], axis=1)
x_test.head(10)


x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [ ]:
model = XGBClassifier(objective='binary:logistic')
model.fit(x_train, y_train)

In [ ]:
y_pred_proba = model.predict_proba(x_test)
y_pred_proba=[i[1] for i in y_pred_proba]
print("XGB ROC score =",roc_auc_score(y_test,y_pred_proba))

In [ ]:
output = x_test
output['Exited'] = y_pred_proba

In [ ]:
columns_to_keep = ['id', 'Exited']

# Find columns to drop
columns_to_drop = [col for col in output.columns if col not in columns_to_keep]

# Drop the columns
output.drop(columns=columns_to_drop, inplace=True)

output.head(5)

In [ ]:
output.to_csv('output/submission.csv', index=False)